In [1]:
# Import MNIST data
import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

('Extracting', '/tmp/data/train-images-idx3-ubyte.gz')
('Extracting', '/tmp/data/train-labels-idx1-ubyte.gz')
('Extracting', '/tmp/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/tmp/data/t10k-labels-idx1-ubyte.gz')


In [3]:
import tensorflow as tf

In [4]:
# Set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [5]:
# TF graph input
x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 classes

In [6]:
# Create a model

In [7]:
# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [8]:
with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [9]:
# Add summary ops to collect data
# w_h = tf.histogram_summary("weights", W)
# b_h = tf.histogram_summary("biases", b)
# tf.histogram_summary has been deprecated.
# Switch to tf.summary.histogram.
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

In [11]:
# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a summary to monitor the cost function
    # tf.scalar_summary is deprecated. Please switch to tf.summary.scalar.
    tf.summary.scalar("cost_function", cost_function)

In [12]:
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [16]:
# Initializing the variables
# init = tf.initialize_all_variables() # deprecated
init = tf.global_variables_initializer()

In [18]:
# Merge all summaries into a single operator
# merged_summary_op = tf.merge_all_summaries()
# tf.merge_all_summaries() is deprecated. Please switch to tf.summary.merge_all.
merged_summary_op = tf.summary.merge_all()

In [23]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Change this to a location on your computer
    # summary_writer = tf.train.SummaryWriter('/ALTA2/sjcho/tf_practice/Siraj/tensorflow_demo/SummaryWriter/', graph_def=sess.graph_def)
    # tf.train.SummaryWriter is deprecated. Switch to tf.summary.FileWriter
    # summary_writer = tf.summary.FileWriter('/ALTA2/sjcho/tf_practice/Siraj/tensorflow_demo/SummaryWriter/', graph_def=sess.graph_def)
    # WARNING:tensorflow:Passing a `GraphDef` to the SummaryWriter is deprecated. Pass a `Graph` object instead, such as `sess.graph`.
    summary_writer = tf.summary.FileWriter('/ALTA2/sjcho/tf_practice/Siraj/tensorflow_demo/SummaryWriter/', graph=sess.graph)
    
    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print "Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost)

    print "Tuning completed!"

    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

Iteration: 0001 cost= 30.799457616
Iteration: 0003 cost= 21.073307491
Iteration: 0005 cost= 20.181585053
Iteration: 0007 cost= 19.534622709
Iteration: 0009 cost= 19.295928841
Iteration: 0011 cost= 19.323360469
Iteration: 0013 cost= 18.725063724
Iteration: 0015 cost= 18.823398877
Iteration: 0017 cost= 18.533824621
Iteration: 0019 cost= 18.574166284
Iteration: 0021 cost= 18.374365897
Iteration: 0023 cost= 18.166347896
Iteration: 0025 cost= 18.288271149
Iteration: 0027 cost= 18.038603953
Iteration: 0029 cost= 17.980189320
Tuning completed!
Accuracy: 0.9197
